## Number Double check (8/26/17)

In [1]:
import io, os, sys, types

from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path
        
class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
            for cell in nb.cells:
                if cell.cell_type == 'code':
                    # transform the input to executable Python
                    code = self.shell.input_transformer_manager.transform_cell(cell.source)
                    # run the code in themodule
                    exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod

    
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

    
sys.meta_path.append(NotebookFinder())


print ('Complete')

Complete


In [2]:
from Masterfunctions_SignalRates import *

print ('All Notebooks Imported')

importing Jupyter notebook from Masterfunctions_SignalRates.ipynb
importing Jupyter notebook from MasterFunctions_Capture.ipynb
importing Jupyter notebook from MasterFunctions_ModelData.ipynb
------ MasterFunctions_ModelData Imported ------
------ MasterFunctions_Capture Loaded ------
importing Jupyter notebook from MasterFunctions_Annihilation.ipynb
------ MasterFunctions_Annihilation Loaded ------
------ MasterFunctions_EventRates Imported ------
All Notebooks Imported


Define the following parameters:
\begin{enumerate}
\item $m_X = 1000$ GeV
\item $m_A=1.0$ GeV
\item $\varepsilon = 10^{-8}$
\item $\alpha = 1/137$
\item $\alpha_X = \alpha_{Th}$
\item BR = 1
\end{enumerate}

In [3]:
print ('We define the following:')
print ('m_X     = 1000.0 Gev')
print ('m_A     = 1.0 GeV')
print ('Epsilon = 10^{-8}')
print ('alpha   = 1/137')
print ('alpha_X = alpha_{Th} = 0.0345505918929')

We define the following:
m_X     = 1000.0 Gev
m_A     = 1.0 GeV
Epsilon = 10^{-8}
alpha   = 1/137
alpha_X = alpha_{Th} = 0.0345505918929


In [4]:
testm_X = 1000.
testm_A = 1.
testEpsilon = 10**-8
testAlpha = 1./137
testAlpha_X = alphaTherm(testm_X,testm_A)
testBR = 1.

print ('Complete')

Complete


Calculate the following
\begin{enumerate}
\item CCap Iron
\item CCap Total
\item ThermAvgSommerfeld
\item CAnn
\item Decay Length
\end{enumerate}

In [5]:
#################
# Capture
#################

CCapAdam0 = {
    'O16' : 0,
    'Si28': 0,
    'Mg24': 0,
    'Fe56': 0,
    'Ca40': 0,
    'P31' : 0,
    'S32' : 0,
    'Al27': 0,
    'Cr52': 0 
}

tempSum = 0
for element in elementList:
    CCapAdam0[element] = singleElementCap(element, testm_X, testm_A, testEpsilon, testAlpha, testAlpha_X)
    tempSum += CCapAdam0[element]
    

capIron = CCapAdam0['Fe56']
capTotal = tempSum

#################
# Annihilation
#################
SigmaVTree = sigmaVtree(testm_X ,testm_A, testAlpha_X)
Sommerfeld = thermAvgSommerfeld(testm_X, testm_A, testAlpha_X)

cann = CAnnCalc(testm_X, testm_A, testAlpha_X, SigmaVTree)
cannSomm = CAnnSommCalc(testm_X, testm_A, testAlpha_X, SigmaVTree, Sommerfeld)

#################
# EQ Time
#################
Tau = tau(capTotal, cann)
TauSomm = tau(capTotal, cannSomm)


#################
# Decay Length
#################
DecayLength = decayLength(testm_X, testm_A, testEpsilon, testBR)

print ('Complete')

Complete


In [6]:
print ('--------------')
print ('CCap Iron  : {0} [sec^-1]'.format(capIron))
print ('CCap Total : {0} [sec^-1]'.format(capTotal))
print ()
print ('Therm Avg Sommerfeld : {0}'.format(Sommerfeld))
print ('CAnn (no Somm)       : {0} [sec^-1]'.format(cann))
print ('CAnn (Somm)          : {0} [sec^-1]'.format(cannSomm))
print ()
print ('Tau (no Sommerfeld) : {0} [sec]'.format(Tau))
print ('Tau (Sommerfeld)    : {0} [sec]'.format(TauSomm))
print ()
print ('Decay Length (BR = 1) : {0} [cm]'.format(DecayLength))
print ('--------------')


--------------
CCap Iron  : 121117517.03176607 [sec^-1]
CCap Total : 138044519.52916643 [sec^-1]

Therm Avg Sommerfeld : 222.07693939153845
CAnn (no Somm)       : 7.474936906482988e-49 [sec^-1]
CAnn (Somm)          : 1.6600111103365965e-46 [sec^-1]

Tau (no Sommerfeld) : 9.844341016146682e+19 [sec]
Tau (Sommerfeld)    : 6.605944457989461e+18 [sec]

Decay Length (BR = 1) : 84700000.0 [cm]
--------------
